In [1]:

import pandas as pd
import numpy as np
import typing

import sys
sys.path.append('..')
import doctable

In [2]:
@doctable.table_schema
class Record:
    name: str = doctable.Column(column_args=doctable.ColumnArgs(nullable=False, unique=True))
    age: int = doctable.Column()
    is_old: bool = doctable.Column()
    
    id: int = doctable.Column(
        column_args=doctable.ColumnArgs(
            order = 0, 
            primary_key=True, 
            autoincrement=True
        ),
    )

core = doctable.ConnectCore.open(target=':memory:', dialect='sqlite', echo=True)

with core.begin_ddl() as ddl:
    rtab = ddl.create_table_if_not_exists(container_type=Record)

2023-11-12 11:37:09,042 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,042 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Record")
2023-11-12 11:37:09,043 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-12 11:37:09,043 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Record")
2023-11-12 11:37:09,044 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-12 11:37:09,045 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Record" (
	id INTEGER, 
	age INTEGER, 
	is_old INTEGER, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2023-11-12 11:37:09,046 INFO sqlalchemy.engine.Engine [no key 0.00039s] ()
2023-11-12 11:37:09,046 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
import random
random.seed(0)

new_records: typing.List[Record] = list()
for i in range(10):
    age = int(random.random()*100) # number in [0,1]
    is_old = age > 50
    new_records.append(Record(name='user_'+str(i), age=age, is_old=is_old))

# insert new records
with rtab.query() as q:
    print(q.insert_multi(new_records))

# dataframe select (for example purposes - .df() will be covered later)
with core.query() as q:
    r = q.select(rtab.all_cols()).df()
r

2023-11-12 11:37:09,086 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,087 INFO sqlalchemy.engine.Engine INSERT OR FAIL INTO "Record" (age, is_old, name) VALUES (?, ?, ?)
2023-11-12 11:37:09,088 INFO sqlalchemy.engine.Engine [generated in 0.00190s] [(84, True, 'user_0'), (75, True, 'user_1'), (42, False, 'user_2'), (25, False, 'user_3'), (51, True, 'user_4'), (40, False, 'user_5'), (78, True, 'user_6'), (30, False, 'user_7'), (47, False, 'user_8'), (58, True, 'user_9')]
2023-11-12 11:37:09,089 INFO sqlalchemy.engine.Engine COMMIT
2023-11-12 11:37:09,092 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,093 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
2023-11-12 11:37:09,093 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ()
2023-11-12 11:37:09,095 INFO sqlalchemy.engine.Engine COMMIT


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1
2,3,42,0,user_2
3,4,25,0,user_3
4,5,51,1,user_4
5,6,40,0,user_5
6,7,78,1,user_6
7,8,30,0,user_7
8,9,47,0,user_8
9,10,58,1,user_9


## Two Interfaces: `ConnectQuery` and `TableQuery`

There are two interfaces for performing queries: `ConnectQuery` and `TableQuery`. 

+ **`ConnectQuery`** table-agnostic interface for querying any table in any result format.

+ **`TableQuery`** table-specific interface for querying a specific table. Insert and select from container objects used to define the schema.


In [4]:
# ConnectQuery - table agnostic
with core.query() as q:
    print(type(q))

# TableQuery - queries are relative to specific 
# table, results appear as container objects
with rtab.query() as q:
    print(type(q))

<class 'doctable.query.connectquery.ConnectQuery'>
<class 'doctable.query.tablequery.TableQuery'>


### `ConnectQuery` Basics

Every query begins with the `.query()` method, which returns a `ConnectQuery` object. This object maintains a database connection, and, when used as a context manager, will commit all changes upon exit. It is fine to use the `ConnectQuery` object without a context manager for queries that do not require commits.

This example is the most basic select query we can execute. Note that `ConnectQuery` methods are table-agnostic, so we must specify columns to be selected - in this case, we provide `rtab.all_cols()` to specify that we want to query all columns from the `Record` table. It returns a `sqlalchemy.CursorResult` object that we will discuss later.

In [5]:
core.query().select(rtab.all_cols())

2023-11-12 11:37:09,196 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,199 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
2023-11-12 11:37:09,200 INFO sqlalchemy.engine.Engine [cached since 0.1082s ago] ()


### Selecting Specific Columns

In many cases, you would not want to select all columns from a given table - for this reason, there are several methods you can use to specify the desired columns. In addition to `.all_cols()` in the above snippet, you may use any of these methods.

| Method | Description |
| --- | --- |
| `.all_cols()` | specify that we want all columns |
| `.cols('col1', 'col2')` | specify set of columns |
| `table['col1']` | specify single column |
| `table[['col1', 'col2']]` | specify multiple columns |
| `table['col1':'col3']` | specify sequential range of columns |

In [6]:
# select all columns
rtab.all_cols()

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('age', Integer(), table=<Record>),
 Column('is_old', Integer(), table=<Record>),
 Column('name', String(), table=<Record>, nullable=False)]

In [7]:
# .cols method
rtab.cols('id', 'name')

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('name', String(), table=<Record>, nullable=False)]

In [8]:
# single-column subscript
rtab['age']

Column('age', Integer(), table=<Record>)

In [9]:
# list of columns
rtab[['id','is_old']]

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('is_old', Integer(), table=<Record>)]

In [10]:
# slice select
rtab['id':'is_old']

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('age', Integer(), table=<Record>),
 Column('is_old', Integer(), table=<Record>)]

Note that the `.select()` method requires a list of columns, so we can combine these methods by combining the lists they return. Obviously, the order matters for the returned values.

In [11]:
rtab.cols('id','is_old') + [rtab['name']]

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('is_old', Integer(), table=<Record>),
 Column('name', String(), table=<Record>, nullable=False)]

The `.select()` method always accepts a list of columns, so be sure to wrap single-column selections in a list.

In [12]:
core.query().select([rtab['name']])

2023-11-12 11:37:09,590 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,592 INFO sqlalchemy.engine.Engine SELECT "Record".name 
FROM "Record"
2023-11-12 11:37:09,594 INFO sqlalchemy.engine.Engine [generated in 0.00375s] ()


In [13]:
core.query().select(rtab.cols('id','is_old') + [rtab['name']])

2023-11-12 11:37:09,646 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,649 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".is_old, "Record".name 
FROM "Record"
2023-11-12 11:37:09,651 INFO sqlalchemy.engine.Engine [generated in 0.00483s] ()


### Working with Query Results

Now we turn to working with the results objects. So far I have demonstrated values for returning `sqlalchemy.CursorResult` objects, but additional methods are required to return the results in a usable format. The following methods are available for various purposes:

| Method | Description |
| --- | --- |
| `result.all()` | return all results in query
| `result.df()` | return multiple results as a dataframe
| `result.first()` | return first result in query
| `result.one()` | return exactly one result in query. NOTE: raises exception if not exactly one result.
| `result.scalar_one()` | return single result, end query. NOTE: raises exception if not exactly one result.
| `result.scalars().all()` | return single column of results

In [14]:
with core.query() as q:
    r = q.select(rtab.all_cols(), limit=3)
r.all()

2023-11-12 11:37:09,703 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,706 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 11:37:09,708 INFO sqlalchemy.engine.Engine [generated in 0.00471s] (3, 0)
2023-11-12 11:37:09,711 INFO sqlalchemy.engine.Engine COMMIT


[(1, 84, 1, 'user_0'), (2, 75, 1, 'user_1'), (3, 42, 0, 'user_2')]

In [15]:
core.query().select(rtab.all_cols(), limit=3).df()

2023-11-12 11:37:09,760 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,763 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 11:37:09,765 INFO sqlalchemy.engine.Engine [cached since 0.06133s ago] (3, 0)


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1
2,3,42,0,user_2


In [16]:
# raises exception without limit=1
core.query().select(rtab.all_cols()).first()

2023-11-12 11:37:09,817 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,819 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
2023-11-12 11:37:09,821 INFO sqlalchemy.engine.Engine [cached since 0.729s ago] ()


(1, 84, 1, 'user_0')

In [17]:
# raises exception if more than one result is returned 
# (here I forced this with limit=1)
core.query().select(rtab.all_cols(), limit=1).one()

2023-11-12 11:37:09,876 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,879 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 11:37:09,881 INFO sqlalchemy.engine.Engine [cached since 0.1774s ago] (1, 0)


(1, 84, 1, 'user_0')

In [18]:
# this returns the first column from the first row, then closes the cursor
core.query().select(rtab.all_cols(), limit=1).scalar_one()

2023-11-12 11:37:09,934 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:09,936 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 11:37:09,938 INFO sqlalchemy.engine.Engine [cached since 0.2351s ago] (1, 0)


1

In [19]:
# it makes more sense to query a single column
core.query().select(rtab.cols('is_old'), limit=1).scalar_one()

2023-11-12 11:37:10,051 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:10,053 INFO sqlalchemy.engine.Engine SELECT "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 11:37:10,054 INFO sqlalchemy.engine.Engine [generated in 0.00259s] (1, 0)


1

In [20]:
# and when when you need a single column, use .scalars() instead of .all()
core.query().select(rtab.cols('is_old')).scalars().all()

2023-11-12 11:37:10,104 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:10,106 INFO sqlalchemy.engine.Engine SELECT "Record".is_old 
FROM "Record"
2023-11-12 11:37:10,107 INFO sqlalchemy.engine.Engine [generated in 0.00294s] ()


[1, 1, 0, 0, 1, 0, 1, 0, 0, 1]

### Conditional Select Statements

| operator | description |
| --- | --- |
| `&`, `doctable.f.and_()` | and |
| `\|`, `doctable.f.or_()` | or |
| `==` | equals |
| `!=`, `doctable.f.not_()` | not equals |
| `>` | greater than |
| `>=` | greater than or equal to |
| `<` | less than |
| `<=` | less than or equal to |
| `in_()` | in list |
| `contains()` | check if item is substring |
| `like()` | like string |
| `ilike()` | case-insensitive like string |
| `between()`, `doctable.f.between()` | between two values |
| `is_()` | is value |
| `isnot()` | is not value |
| `startswith()` | starts with string |

In [21]:
core.query().select(rtab.all_cols(), where=rtab['id']==2).df()

2023-11-12 11:37:10,158 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:10,161 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id = ?
2023-11-12 11:37:10,163 INFO sqlalchemy.engine.Engine [generated in 0.00458s] (2,)


,id,age,is_old,name
0,2,75,1,user_1


In [41]:
core.query().select(rtab.all_cols(), where=rtab['id']<rtab['id']).df()

2023-11-12 11:43:23,097 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:43:23,098 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id < "Record".id
2023-11-12 11:43:23,099 INFO sqlalchemy.engine.Engine [generated in 0.00164s] ()


""


In [23]:
core.query().select(rtab.all_cols(), where=(rtab['id']%2)==0).df()

2023-11-12 11:37:10,289 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:10,290 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id % ? = ?
2023-11-12 11:37:10,290 INFO sqlalchemy.engine.Engine [generated in 0.00172s] (2, 0)


,id,age,is_old,name
0,2,75,1,user_1
1,4,25,0,user_3
2,6,40,0,user_5
3,8,30,0,user_7
4,10,58,1,user_9


In [24]:
condition = (rtab['id']>=2) & (rtab['id']<=4) & (rtab['name']!='user_2')
core.query().select(rtab.all_cols(), where=condition).df()

2023-11-12 11:37:10,340 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:10,341 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id >= ? AND "Record".id <= ? AND "Record".name != ?
2023-11-12 11:37:10,341 INFO sqlalchemy.engine.Engine [generated in 0.00175s] (2, 4, 'user_2')


,id,age,is_old,name
0,2,75,1,user_1
1,4,25,0,user_3


In [25]:
condition = rtab['name'].in_(('user_2','user_3'))
core.query().select(rtab.all_cols(), where=condition).df()

2023-11-12 11:37:10,395 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:10,396 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".name IN (?, ?)
2023-11-12 11:37:10,396 INFO sqlalchemy.engine.Engine [generated in 0.00167s] ('user_2', 'user_3')


,id,age,is_old,name
0,3,42,0,user_2
1,4,25,0,user_3


In [26]:
core.query().select(rtab.all_cols(), where=rtab['id'].between(2,4)).df()

2023-11-12 11:37:10,449 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:10,450 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id BETWEEN ? AND ?
2023-11-12 11:37:10,451 INFO sqlalchemy.engine.Engine [generated in 0.00209s] (2, 4)


,id,age,is_old,name
0,2,75,1,user_1
1,3,42,0,user_2
2,4,25,0,user_3


In [27]:
condition = ~(rtab['name'].in_(('user_2','user_3'))) & (rtab['id'] < 4)
core.query().select(rtab.all_cols(), where=condition).df()

2023-11-12 11:37:10,505 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:10,506 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE ("Record".name NOT IN (?, ?)) AND "Record".id < ?
2023-11-12 11:37:10,507 INFO sqlalchemy.engine.Engine [generated in 0.00174s] ('user_2', 'user_3', 4)


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1


In [47]:
condition = doctable.f.or_(doctable.f.not_(rtab['id']==4)) & (rtab['id'] <= 2)
core.query().select(rtab.all_cols(), where=condition).df()

2023-11-12 11:47:01,803 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:47:01,806 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id != ? AND "Record".id <= ?
2023-11-12 11:47:01,808 INFO sqlalchemy.engine.Engine [cached since 591.2s ago] (4, 2)


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1


In [29]:
with core.query() as q:
    ages = q.select([rtab['age']]).scalars().all()
    mean_age = sum(ages)/len(ages)
    result = q.select(rtab.all_cols(), where=rtab['age']>mean_age)
result.df()

2023-11-12 11:37:10,615 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:37:10,617 INFO sqlalchemy.engine.Engine SELECT "Record".age 
FROM "Record"
2023-11-12 11:37:10,618 INFO sqlalchemy.engine.Engine [generated in 0.00266s] ()
2023-11-12 11:37:10,619 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 11:37:10,620 INFO sqlalchemy.engine.Engine [generated in 0.00062s] (53.0,)
2023-11-12 11:37:10,621 INFO sqlalchemy.engine.Engine COMMIT


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1
2,7,78,1,user_6
3,10,58,1,user_9


### Column Operators

In addition to any of the methods used for conditional selects, there are several additional methods that can be used to transform columns in the select statement.

| Method | Description |
| --- | --- |
| `.label()` | rename column in result (particularly useful after transformations) |
| `.min()`, `doctable.f.min()` | max of column values |
| `.max()`, `doctable.f.max()` | max of column values |
| `.sum()`, `doctable.f.sum()` | sum of column |
| `.count()`, `doctable.f.count()` | count number of results. *NOTE*: must use `f.count()` when counting transformed columns. |
| `.distinct()`, `doctable.f.distinct()` | get distinct values |
| `/` | divide |
| `*` | multiply |
| `+` | add |
| `-` | subtract |
| `%` | modulo |
| `.concat()` | concatenate strings |

In [33]:
columns = [
    (rtab['id'] % 2).label('mod_id'), 
    rtab['name'].label('myname')
]
core.query().select(columns, where=rtab['is_old']).df()

2023-11-12 11:40:35,446 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:40:35,447 INFO sqlalchemy.engine.Engine SELECT "Record".id % ? AS mod_id, "Record".name AS myname 
FROM "Record" 
WHERE "Record".is_old
2023-11-12 11:40:35,447 INFO sqlalchemy.engine.Engine [generated in 0.00161s] (2,)


,mod_id,myname
0,1,user_0
1,0,user_1
2,1,user_4
3,1,user_6
4,0,user_9


In [48]:
formula = rtab['age'].sum() / rtab['age'].count()
core.query().select([formula]).scalar_one()

2023-11-12 11:48:38,089 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:48:38,091 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS anon_1 
FROM "Record"
2023-11-12 11:48:38,093 INFO sqlalchemy.engine.Engine [cached since 390.2s ago] ()


Decimal('53.0000000000')

In [49]:
formula = rtab['age'].max() - rtab['age'].min()
core.query().select([formula]).scalar_one()

2023-11-12 11:50:05,558 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:50:05,560 INFO sqlalchemy.engine.Engine SELECT max("Record".age) - min("Record".age) AS anon_1 
FROM "Record"
2023-11-12 11:50:05,562 INFO sqlalchemy.engine.Engine [generated in 0.00443s] ()


59

In [51]:
# average age of individuals over 30
formula = rtab['age'].sum() / rtab['age'].count()
condition = rtab['age'] > 30
core.query().select([formula], where=condition).scalar_one()

2023-11-12 11:51:37,862 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 11:51:37,864 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS anon_1 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 11:51:37,865 INFO sqlalchemy.engine.Engine [cached since 18.62s ago] (30,)


Decimal('59.3750000000')

In [75]:
# descriptive stats on age of individuals over 30
columns = [
    (rtab['age'].sum() / rtab['age'].count()).label('mean'),
    rtab['age'].max().label('max'),
    rtab['age'].min().label('min'),
]
condition = rtab['age'] > 30
core.query().select(columns, where=condition).df()

2023-11-12 12:12:22,524 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 12:12:22,525 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS mean, max("Record".age) AS max, min("Record".age) AS min 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 12:12:22,525 INFO sqlalchemy.engine.Engine [cached since 77.72s ago] (30,)


,mean,max,min
0,59.3750000000,84,40


In [76]:
# all distinct values
formula = rtab['is_old'].distinct()
core.query().select([formula]).df()

2023-11-12 12:12:35,604 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 12:12:35,607 INFO sqlalchemy.engine.Engine SELECT distinct("Record".is_old) AS distinct_1 
FROM "Record"
2023-11-12 12:12:35,608 INFO sqlalchemy.engine.Engine [cached since 52.36s ago] ()


,distinct_1
0,1
1,0


In [74]:
# count individuals over 30
core.query().select([doctable.f.count()], where=rtab['age']>30).scalar_one()

2023-11-12 12:12:07,615 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 12:12:07,618 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 12:12:07,619 INFO sqlalchemy.engine.Engine [cached since 227.3s ago] (30,)


8

In [77]:
# similar to previous
core.query().select([rtab['id'].count()], where=rtab['age']>30).scalar_one()

2023-11-12 12:13:23,418 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 12:13:23,420 INFO sqlalchemy.engine.Engine SELECT count("Record".id) AS count_1 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 12:13:23,421 INFO sqlalchemy.engine.Engine [generated in 0.00284s] (30,)


8